In [1]:
import os

from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 13


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674
4,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-10-05T19:34:13-05:00 -> 1696552453000, Until: 2023-10-05T19:39:13-05:00 -> 1696552753000


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp')
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,
2023-10-05 19:34:30-05:00,-147.2,30.76,30.02,30.39,28.49,30.81,31.19,31.34,28.71,30.8,29.1,31.71,31.40
2023-10-05 19:35:00-05:00,-147.2,30.74,29.96,30.39,28.53,30.81,31.21,31.37,28.68,30.7,29.2,31.74,31.38
2023-10-05 19:35:30-05:00,-147.2,30.74,29.97,30.41,28.57,30.78,31.19,31.36,28.67,30.8,29.0,31.70,31.39
2023-10-05 19:36:00-05:00,-147.2,30.74,29.97,30.45,28.64,30.81,31.24,31.33,28.70,30.7,29.1,31.73,31.39
2023-10-05 19:36:30-05:00,-147.2,30.72,29.95,30.42,28.60,30.81,31.25,31.34,28.64,30.8,29.1,31.73,31.39


## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,10.0,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,10.000000,10.000000,9.000000,10.000000,10.000000,10.000000
mean,-147.2,30.702222,29.946667,30.409000,28.589000,30.801111,31.203333,31.319000,28.625000,30.777778,29.070000,31.737000,31.387000
std,0.0,0.049441,0.041533,0.019692,0.047246,0.013642,0.030414,0.036347,0.066542,0.044096,0.067495,0.020575,0.011595
min,-147.2,30.630000,29.890000,30.390000,28.490000,30.780000,31.150000,31.260000,28.520000,30.700000,29.000000,31.700000,31.360000
25%,-147.2,30.670000,29.920000,30.390000,28.577500,30.790000,31.190000,31.300000,28.580000,30.800000,29.000000,31.730000,31.382500
50%,-147.2,30.720000,29.950000,30.410000,28.600000,30.810000,31.190000,31.325000,28.625000,30.800000,29.100000,31.740000,31.390000
75%,-147.2,30.740000,29.970000,30.420000,28.610000,30.810000,31.220000,31.340000,28.677500,30.800000,29.100000,31.755000,31.390000
max,-147.2,30.760000,30.020000,30.450000,28.640000,30.820000,31.250000,31.370000,28.710000,30.800000,29.200000,31.760000,31.400000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
if not is_ok:
    print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")